In [1]:
!pip install azure-ai-formrecognizer

  Using cached https://files.pythonhosted.org/packages/44/f0/d7ea506b59a6a4040f833b89e0e39c0eaf6b8ff9a7844124b7a4ed22c255/azure_ai_formrecognizer-3.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/9c/8c/6463dc620629d71c02f3f1a7dc30aab8e4f1a08ef64726cc3dd4f7949096/azure_core-1.8.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/fa/f5/9e315fe8cb985b0ce052b34bcb767883dc739f46fadb62f05a7e6d6eedbe/msrest-0.6.19-py2.py3-none-any.whl
  Found existing installation: azure-core 1.3.0
    Uninstalling azure-core-1.3.0:
      Successfully uninstalled azure-core-1.3.0
  Found existing installation: msrest 0.6.11
    Uninstalling msrest-0.6.11:
      Successfully uninstalled msrest-0.6.11


ERROR: botframework-connector 4.0.0a6 has requirement msrest==0.4.29, but you'll have msrest 0.6.19 which is incompatible.
ERROR: botbuilder-schema 4.0.0a6 has requirement msrest==0.4.29, but you'll have msrest 0.6.19 which is incompatible.


In [3]:
from azure.ai.formrecognizer import FormRecognizerClient, FormTrainingClient
from azure.core.credentials import AzureKeyCredential
import json

config = json.load(open("config.json"))

In [6]:
client = FormRecognizerClient(config["endpoint"], AzureKeyCredential(config["key"]))

## Reading Receipts

[Sample](https://raw.githubusercontent.com/Azure/azure-sdk-for-python/master/sdk/formrecognizer/azure-ai-formrecognizer/tests/sample_forms/receipt/contoso-allinone.jpg)

In [7]:
receiptUrl = "https://raw.githubusercontent.com/Azure/azure-sdk-for-python/master/sdk/formrecognizer/azure-ai-formrecognizer/tests/sample_forms/receipt/contoso-allinone.jpg"

In [8]:
receipt_recognizer = client.begin_recognize_receipts_from_url(receiptUrl)

result = receipt_recognizer.result()

In [9]:
for receipt in result:
    for name, field in receipt.fields.items():
        if name == "Items":
            print("Receipt Items:")
            for idx, items in enumerate(field.value):
                print("...Item #{}".format(idx + 1))
                for item_name, item in items.value.items():
                    print("......{}: {} has confidence {}".format(item_name, item.value, item.confidence))
        else:
            print("{}: {} has confidence {}".format(name, field.value, field.confidence))

ReceiptType: Itemized has confidence 0.692
MerchantName: Contoso Contoso has confidence 0.613
MerchantAddress: 123 Main Street Redmond, WA 98052 has confidence 0.99
MerchantPhoneNumber: +19876543210 has confidence 0.99
TransactionDate: 2019-06-10 has confidence 0.99
TransactionTime: 13:59:00 has confidence 0.977
Receipt Items:
...Item #1
......Quantity: 1.0 has confidence 0.92
......Name: Cappuccino has confidence 0.923
......TotalPrice: 2.2 has confidence 0.918
...Item #2
......Quantity: 1.0 has confidence 0.858
......Name: BACON & EGGS has confidence 0.916
......TotalPrice: 9.5 has confidence 0.916
Subtotal: 11.7 has confidence 0.955
Tax: 1.17 has confidence 0.979
Tip: 1.63 has confidence 0.941
Total: 14.5 has confidence 0.985


## Reading Forms

[Sample](https://raw.githubusercontent.com/Azure/azure-sdk-for-python/master/sdk/formrecognizer/azure-ai-formrecognizer/tests/sample_forms/forms/Invoice_1.pdf)

In [17]:
sample_form = "https://raw.githubusercontent.com/Azure/azure-sdk-for-python/master/sdk/formrecognizer/azure-ai-formrecognizer/tests/sample_forms/forms/Invoice_1.pdf"

In [20]:
form_recognizer = client.begin_recognize_content_from_url(sample_form)

result = form_recognizer.result()

In [21]:
page = result[0].tables[0]

page

FormTable(page_number=1, cells=[FormTableCell(text=Invoice Number, row_index=0, column_index=0, row_span=1, column_span=1, bounding_box=[Point(x=0.5075, y=2.8088), Point(x=1.9061, y=2.8088), Point(x=1.9061, y=3.3219), Point(x=0.5075, y=3.3219)], confidence=1.0, is_header=False, is_footer=False, page_number=1, field_elements=[FormWord(text=Invoice, bounding_box=[Point(x=0.5439, y=2.8733), Point(x=1.0098, y=2.8733), Point(x=1.0098, y=2.9754), Point(x=0.5439, y=2.9754)], confidence=1.0, page_number=1, kind=word), FormWord(text=Number, bounding_box=[Point(x=1.0611, y=2.8743), Point(x=1.5729, y=2.8743), Point(x=1.5729, y=2.9754), Point(x=1.0611, y=2.9754)], confidence=1.0, page_number=1, kind=word)]), FormTableCell(text=Invoice Date, row_index=0, column_index=1, row_span=1, column_span=1, bounding_box=[Point(x=1.9061, y=2.8088), Point(x=3.3074, y=2.8088), Point(x=3.3074, y=3.3219), Point(x=1.9061, y=3.3219)], confidence=1.0, is_header=False, is_footer=False, page_number=1, field_elements=[F

In [26]:
for item in page.cells:
    if (item.row_index == 0):
        print("Row Name:")
        print(f"Text: {item.text} - Location: {item.bounding_box} - Confidence {item.confidence}")
    else:
        print("...Item:")
        print(f"...Text: {item.text} - Location: {item.bounding_box} - Confidence {item.confidence}")

Row Name:
Text: Invoice Number - Location: [Point(x=0.5075, y=2.8088), Point(x=1.9061, y=2.8088), Point(x=1.9061, y=3.3219), Point(x=0.5075, y=3.3219)] - Confidence 1.0
Row Name:
Text: Invoice Date - Location: [Point(x=1.9061, y=2.8088), Point(x=3.3074, y=2.8088), Point(x=3.3074, y=3.3219), Point(x=1.9061, y=3.3219)] - Confidence 1.0
Row Name:
Text: Invoice Due Date - Location: [Point(x=3.3074, y=2.8088), Point(x=4.7074, y=2.8088), Point(x=4.7074, y=3.3219), Point(x=3.3074, y=3.3219)] - Confidence 1.0
Row Name:
Text: Charges - Location: [Point(x=4.7074, y=2.8088), Point(x=5.386, y=2.8088), Point(x=5.386, y=3.3219), Point(x=4.7074, y=3.3219)] - Confidence 1.0
Row Name:
Text: VAT ID - Location: [Point(x=6.1051, y=2.8088), Point(x=7.5038, y=2.8088), Point(x=7.5038, y=3.3219), Point(x=6.1051, y=3.3219)] - Confidence 1.0
...Item:
...Text: 34278587 - Location: [Point(x=0.5075, y=3.3219), Point(x=1.9061, y=3.3219), Point(x=1.9061, y=3.859), Point(x=0.5075, y=3.859)] - Confidence 1.0
...Item:


## Reading Business Cards

In [33]:
from requests import get, post

In [13]:
endpoint = config['endpoint']

In [14]:
url = f"{endpoint}/formrecognizer/v2.1-preview.1/prebuilt/businessCard/analyze"

In [15]:
headers = {
    "Content-Type": "image/png",
    "Ocp-Apim-Subscription-Key": config["key"]
}

In [18]:
params = {
    "includeTextDetails": True
}

In [19]:
with open("businesscard-sample.png", "rb") as f:
    data = f.read()

In [23]:
response = post(url, data=data, headers=headers, params=params)

In [32]:
if response.status_code != 202:
        print("POST analyze failed:\n%s" % response.text)

print("POST analyze succeeded:\n%s" % response.headers)

POST analyze succeeded:
{'Content-Length': '0', 'Operation-Location': 'https://customformrecognizer.cognitiveservices.azure.com/formrecognizer/v2.1-preview.1/prebuilt/businessCard/analyzeResults/89c25e7d-2af0-40e0-8fec-db55099baa01', 'x-envoy-upstream-service-time': '94', 'apim-request-id': '89c25e7d-2af0-40e0-8fec-db55099baa01', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'x-content-type-options': 'nosniff', 'Date': 'Tue, 13 Oct 2020 07:20:41 GMT'}


In [35]:
get_url = response.headers["operation-location"]
get_url

'https://customformrecognizer.cognitiveservices.azure.com/formrecognizer/v2.1-preview.1/prebuilt/businessCard/analyzeResults/89c25e7d-2af0-40e0-8fec-db55099baa01'

In [36]:
get_response = get(get_url, headers = { "Ocp-Apim-Subscription-Key": config["key"] })

In [37]:
get_response.json()

{'status': 'succeeded',
 'createdDateTime': '2020-10-13T07:20:42Z',
 'lastUpdatedDateTime': '2020-10-13T07:20:43Z',
 'analyzeResult': {'version': '2.1.0',
  'readResults': [{'page': 1,
    'angle': 0.1484,
    'width': 521,
    'height': 338,
    'unit': 'pixel',
    'lines': [{'text': 'John Doe',
      'boundingBox': [101, 70, 213, 70, 213, 95, 101, 94],
      'words': [{'text': 'John',
        'boundingBox': [103, 70, 157, 72, 157, 95, 102, 94],
        'confidence': 0.959},
       {'text': 'Doe',
        'boundingBox': [163, 72, 212, 71, 213, 95, 163, 95],
        'confidence': 0.958}]},
     {'text': 'Manager',
      'boundingBox': [101, 101, 175, 102, 175, 120, 100, 118],
      'words': [{'text': 'Manager',
        'boundingBox': [101, 101, 176, 103, 175, 120, 101, 118],
        'confidence': 0.959}]},
     {'text': 'Phone: +41 11 555 43 45',
      'boundingBox': [255, 151, 421, 150, 421, 165, 255, 166],
      'words': [{'text': 'Phone:',
        'boundingBox': [256, 152, 301, 152